In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
%load_ext tensorboard

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [4]:
from PIL import Image
from IPython.display import display

import os
import numpy as np
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf
from tensorflow.keras import layers, models

from get_frames import get_frames
from build_model_ed import build_model_1, build_model_2
from build_model_lstm import build_model_lstm_1, build_model_lstm_2
from create_encoder_decoder_data import process_image, process_dataset, get_full_path
from get_encoder_decoder import get_encoder_decoder

In [5]:
#gpu check
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [50]:
#load data
unit_numb = 100
test_size = 0.1
stride_step = 50
norm_factor=1
dtype = 'uint8'
file_name = f'save_data//encoded_model_2//encoded_{unit_numb}_{stride_step}_ts_{test_size}_{dtype}_norm_{norm_factor}.pkl'
with open(file_name, 'rb') as f:
    frames_data_encoded = pickle.load(f)
    
print(f'Data shape: {frames_data_encoded.shape}')

Data shape: (770, 80, 100)


In [51]:
#shuffle
np.random.shuffle(frames_data_encoded)

In [52]:
#create dataset for lstm with an offset
x_train_lstm = frames_data_encoded[:,:-1,:]
y_train_lstm = frames_data_encoded[:,1:,:]
print(f'x_train_lstm: {x_train_lstm.shape}')
print(f'y_train_lstm: {y_train_lstm.shape}')

x_train_lstm: (770, 79, 100)
y_train_lstm: (770, 79, 100)


In [53]:
cells_list = [200]
ifDense = False
input_shape = (None, unit_numb)
loss = 'mean_squared_error'
learning_rate = 0.1
optimizer_name = "Adadelta"
optimizer = tf.keras.optimizers.legacy.Adadelta(learning_rate=learning_rate, name=optimizer_name)
#optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

model = build_model_lstm_1(input_shape, cells_list, ifDense=ifDense, ifDropout=False)
model.compile(loss=loss, 
              optimizer=optimizer, )
              #metrics=metrics)
model.summary() 

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, None, 100)]       0         
                                                                 
 lstm_7 (LSTM)               (None, None, 200)         240800    
                                                                 
 conv1d_7 (Conv1D)           (None, None, 100)         60100     
                                                                 
Total params: 300900 (1.15 MB)
Trainable params: 300900 (1.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [54]:
#check activation function of the lstm layer
model.layers[1].activation

<function keras.src.activations.tanh(x)>

In [55]:
#model training 16 ifDense False bp 

folder = 'lstm'
epochs = 2000
batch_size = 2 
validation_split = 0.1

cells_list_str = '_'.join(str(x) for x in cells_list)

model_name = f'model_lstm_units_{unit_numb}_bs2_{input_shape[-1]}_{cells_list_str}_ifDense_{ifDense}'

#define callbacks
# Write TensorBoard logs
log_dir = f'./tensorboard/{folder}/logs/{model_name}'
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
#Stop training when no improvement
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                  patience=10, 
                                                  restore_best_weights=True)
#Reduce learning rate when a metric has stopped improving
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=5)
callbacks = [tensorboard, early_stopping, reduce_lr]

In [51]:
# Clear any logs from previous runs
if 1:
    ld = log_dir
else:
    ld=None
!rm -rf $ld

In [56]:
model.fit(
    x_train_lstm,
    y_train_lstm,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split,
    callbacks=callbacks,
)

#model.save_weights(f'saved_models/{folder}/{model_name}/{model_name}')

Epoch 1/2000
347/347 [==============================] - 3s 5ms/step - loss: 0.2428 - val_loss: 0.2346 - lr: 0.1000
Epoch 2/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.2295 - val_loss: 0.2221 - lr: 0.1000
Epoch 3/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.2167 - val_loss: 0.2088 - lr: 0.1000
Epoch 4/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.2048 - val_loss: 0.1982 - lr: 0.1000
Epoch 5/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.1957 - val_loss: 0.1902 - lr: 0.1000
Epoch 6/2000
347/347 [==============================] - 2s 5ms/step - loss: 0.1884 - val_loss: 0.1835 - lr: 0.1000
Epoch 7/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.1823 - val_loss: 0.1777 - lr: 0.1000
Epoch 8/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.1769 - val_loss: 0.1728 - lr: 0.1000
Epoch 9/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.17

Epoch 72/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0722 - val_loss: 0.0714 - lr: 0.1000
Epoch 73/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0715 - val_loss: 0.0707 - lr: 0.1000
Epoch 74/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0708 - val_loss: 0.0700 - lr: 0.1000
Epoch 75/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0701 - val_loss: 0.0693 - lr: 0.1000
Epoch 76/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0693 - val_loss: 0.0686 - lr: 0.1000
Epoch 77/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0686 - val_loss: 0.0679 - lr: 0.1000
Epoch 78/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0680 - val_loss: 0.0673 - lr: 0.1000
Epoch 79/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0673 - val_loss: 0.0666 - lr: 0.1000
Epoch 80/2000
347/347 [==============================] - 1s 4ms/step - l

347/347 [==============================] - 1s 4ms/step - loss: 0.0384 - val_loss: 0.0383 - lr: 0.1000
Epoch 143/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0381 - val_loss: 0.0380 - lr: 0.1000
Epoch 144/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0378 - val_loss: 0.0377 - lr: 0.1000
Epoch 145/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0375 - val_loss: 0.0374 - lr: 0.1000
Epoch 146/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0372 - val_loss: 0.0371 - lr: 0.1000
Epoch 147/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0369 - val_loss: 0.0368 - lr: 0.1000
Epoch 148/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0366 - val_loss: 0.0365 - lr: 0.1000
Epoch 149/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0364 - val_loss: 0.0363 - lr: 0.1000
Epoch 150/2000
347/347 [==============================] - 1s 4ms/step - loss: 0

347/347 [==============================] - 1s 4ms/step - loss: 0.0233 - val_loss: 0.0233 - lr: 0.1000
Epoch 213/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0231 - val_loss: 0.0232 - lr: 0.1000
Epoch 214/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0230 - val_loss: 0.0231 - lr: 0.1000
Epoch 215/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0229 - val_loss: 0.0229 - lr: 0.1000
Epoch 216/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0227 - val_loss: 0.0228 - lr: 0.1000
Epoch 217/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0226 - val_loss: 0.0226 - lr: 0.1000
Epoch 218/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0224 - val_loss: 0.0225 - lr: 0.1000
Epoch 219/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0223 - val_loss: 0.0224 - lr: 0.1000
Epoch 220/2000
347/347 [==============================] - 1s 4ms/step - loss: 0

347/347 [==============================] - 1s 4ms/step - loss: 0.0157 - val_loss: 0.0158 - lr: 0.1000
Epoch 283/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0157 - val_loss: 0.0158 - lr: 0.1000
Epoch 284/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0156 - val_loss: 0.0157 - lr: 0.1000
Epoch 285/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0155 - val_loss: 0.0156 - lr: 0.1000
Epoch 286/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0154 - val_loss: 0.0155 - lr: 0.1000
Epoch 287/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0154 - val_loss: 0.0155 - lr: 0.1000
Epoch 288/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0153 - val_loss: 0.0154 - lr: 0.1000
Epoch 289/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0152 - val_loss: 0.0153 - lr: 0.1000
Epoch 290/2000
347/347 [==============================] - 1s 4ms/step - loss: 0

347/347 [==============================] - 1s 4ms/step - loss: 0.0115 - val_loss: 0.0117 - lr: 0.1000
Epoch 353/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0115 - val_loss: 0.0116 - lr: 0.1000
Epoch 354/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0115 - val_loss: 0.0116 - lr: 0.1000
Epoch 355/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0114 - val_loss: 0.0115 - lr: 0.1000
Epoch 356/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0114 - val_loss: 0.0115 - lr: 0.1000
Epoch 357/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0113 - val_loss: 0.0114 - lr: 0.1000
Epoch 358/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0113 - val_loss: 0.0114 - lr: 0.1000
Epoch 359/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0112 - val_loss: 0.0114 - lr: 0.1000
Epoch 360/2000
347/347 [==============================] - 1s 4ms/step - loss: 0

347/347 [==============================] - 1s 4ms/step - loss: 0.0090 - val_loss: 0.0091 - lr: 0.1000
Epoch 423/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0090 - val_loss: 0.0091 - lr: 0.1000
Epoch 424/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0089 - val_loss: 0.0091 - lr: 0.1000
Epoch 425/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0089 - val_loss: 0.0090 - lr: 0.1000
Epoch 426/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0089 - val_loss: 0.0090 - lr: 0.1000
Epoch 427/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0088 - val_loss: 0.0090 - lr: 0.1000
Epoch 428/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0088 - val_loss: 0.0090 - lr: 0.1000
Epoch 429/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0088 - val_loss: 0.0089 - lr: 0.1000
Epoch 430/2000
347/347 [==============================] - 1s 4ms/step - loss: 0

347/347 [==============================] - 1s 4ms/step - loss: 0.0073 - val_loss: 0.0075 - lr: 0.1000
Epoch 493/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0073 - val_loss: 0.0075 - lr: 0.0100
Epoch 494/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0073 - val_loss: 0.0075 - lr: 0.0100
Epoch 495/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0073 - val_loss: 0.0075 - lr: 0.0100
Epoch 496/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0073 - val_loss: 0.0075 - lr: 0.0100
Epoch 497/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0073 - val_loss: 0.0074 - lr: 0.0100
Epoch 498/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0073 - val_loss: 0.0074 - lr: 0.0100
Epoch 499/2000
347/347 [==============================] - 1s 4ms/step - loss: 0.0073 - val_loss: 0.0074 - lr: 1.0000e-03
Epoch 500/2000
347/347 [==============================] - 1s 4ms/step - los

In [57]:
model.save_weights(f'saved_models/{folder}/{model_name}/{model_name}')

In [49]:
#transfer learning
#the whole model
model.trainable = False

input_layer = layers.Input(shape=input_shape)
x = model(input_layer, training=False)
x = layers.LSTM(40, return_sequences=True)(x)

filters = input_shape[-1]
output_layer = layers.Conv1D(
    filters=filters,
    kernel_size=3,
    activation="sigmoid",
    padding="same",
    data_format='channels_last'
)(x)

model_new = tf.keras.Model(inputs=input_layer, outputs=output_layer)

model_new.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, None, 5)]         0         
                                                                 
 model_3 (Functional)        (None, None, 5)           795       
                                                                 
 lstm_9 (LSTM)               (None, None, 40)          7360      
                                                                 
 conv1d_9 (Conv1D)           (None, None, 5)           605       
                                                                 
Total params: 8760 (34.22 KB)
Trainable params: 7965 (31.11 KB)
Non-trainable params: 795 (3.11 KB)
_________________________________________________________________


In [53]:
model.layers[1]

In [77]:
#transfer learning
#the whole model

input_layer = layers.Input(shape=input_shape)
trained_layer = model.layers[1](input_layer)
trained_layer.trainable = False
new_lstm_layer = layers.LSTM(10, return_sequences=True, activation="relu")(trained_layer)

filters = input_shape[-1]
output_layer = layers.Conv1D(
    filters=filters,
    kernel_size=3,
    activation="sigmoid",
    padding="same",
    data_format='channels_last'
)(new_lstm_layer)

model_new = tf.keras.Model(inputs=input_layer, outputs=output_layer)

model_new.summary()

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, None, 5)]         0         
                                                                 
 lstm_3 (LSTM)               (None, None, 10)          640       
                                                                 
 lstm_17 (LSTM)              (None, None, 10)          840       
                                                                 
 conv1d_17 (Conv1D)          (None, None, 5)           155       
                                                                 
Total params: 1635 (6.39 KB)
Trainable params: 995 (3.89 KB)
Non-trainable params: 640 (2.50 KB)
_________________________________________________________________


In [78]:
loss = 'mean_squared_error'
learning_rate = 0.1
optimizer_name = "Adadelta"
optimizer = tf.keras.optimizers.legacy.Adadelta(learning_rate=learning_rate, name=optimizer_name)

model_new.compile(
    loss=loss, 
    optimizer=optimizer 
)

In [79]:
#model training 16 ifDense False bp 

folder = 'lstm'
epochs = 2000
batch_size = 2 
validation_split = 0.1

model_name = f'model_lstm_tl2_units_{unit_numb}_bs2_{input_shape[-1]}_ifDense_{ifDense}'

#define callbacks
# Write TensorBoard logs
log_dir = f'./tensorboard/{folder}/logs/{model_name}'
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
#Stop training when no improvement
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                  patience=10, 
                                                  restore_best_weights=True)
#Reduce learning rate when a metric has stopped improving
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=5)
callbacks = [tensorboard, early_stopping, reduce_lr]

In [80]:
model_new.fit(
    x_train_lstm,
    y_train_lstm,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split,
    callbacks=callbacks,
)

Epoch 1/2000
347/347 [==============================] - 14s 37ms/step - loss: 0.0429 - val_loss: 0.0402 - lr: 0.1000
Epoch 2/2000
347/347 [==============================] - 13s 36ms/step - loss: 0.0405 - val_loss: 0.0379 - lr: 0.1000
Epoch 3/2000
231/347 [==================>...........] - ETA: 4s - loss: 0.0392

KeyboardInterrupt: 

In [58]:
#tensorboard results
folder = 'lstm'

model_name = f'model_lstm_tl2_units_{unit_numb}_bs2_{input_shape[-1]}_ifDense_{ifDense}'

log_dir = f'./tensorboard/{folder}/logs/{model_name}'
%tensorboard --logdir $log_dir

Reusing TensorBoard on port 6008 (pid 85576), started 0:22:32 ago. (Use '!kill 85576' to kill it.)

In [17]:
#tensorboard results
folder = 'lstm'
cells_list = [10]

model_name = f'model_lstm_units_{unit_numb}_bs2_{input_shape[-1]}_{cells_list_str}_ifDense_{ifDense}'

log_dir = f'./tensorboard/{folder}/logs/{model_name}'
%tensorboard --logdir $log_dir